# Evidence for water worlds around M dwarfs
## Water-world means following Zeng et al. (2019)

**Author: [Hannu Parviainen](mailto:hpparvi@gmail.com)** </br>
**Last edited: 25. October 2023**

### Introduction

Here we study whether our empirical radius and mass estimates support the hypothesis that water worlds exist as a separate individual population between the rocky planets and sub-Neptunes. We do this by calculating Bayesian evidences for three hypotheses stating that the radius and mass distribution for the known planets should be modelled either as a mixture of H$_0$) rocky planets and sub-Neptunes, or H$_1$) rocky planets, a weak mixed population of water-rich planets, and sub-Neptunes, or H$_2$)  rocky planets, a strong population of water-rich planets, and sub-Neptunes, and then calculating the Bayes factor between the hypotheses.

This first notebook uses the theoretical radius-density models by Zeng et al. (2019) to represent the density mean function for water-rich planets. A version of the analysis using the Aguichine et al. (2021) models can be found from the `01b_stpm_aguinchine.ipynb` notebook.

We calculate the Bayesian evidence using nested sampling with the dynasty package.

In [1]:
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
from dynesty import NestedSampler
from uncertainties import ufloat

from spright import RMEstimator
from spright.io import read_stpm
from spright.model import lnlikelihood_sample

%run common.py

### Set up the hypotheses

We set up three competing hypotheses:

 - H$_0$: water-rich planets create a significant population between rocky planets and sub-Neptunes,
 - H$_1$: water-rich planets exist as a mixed population between rocky planets and sub-Neptunes
 - H$_2$: water-rich planets do not exist as a distinct composition population.

These two hypotheses can be presented by different priors on the RDM model water-world population width, $w_\mathrm{w}$. For H$_0$, we remove the water-world population by forcing $w_\mathrm{w}\approx 0$. We encode H$_1$ as $0.15 < w_\mathrm{w} < 0.5$, that is, the water-world population exists but is always mixed with rocky planets and sub-Neputnes. Finally, we encode H$_2$ as $w_\mathrm{w} > 0.5$, that is, the water-world population weight reaches unity at least at one point in the planet radius space. The priors for all the other parameters are the same for the two hypotheses.

We can set these priors in dynesty by creating two different prior transforms that transform the unit cube sampled by dynesty into the model parameter space. 

In [ ]:
def transform_h0(u):
    return transform_base(u, 0.0, 0.001, water_model='z19')

def transform_h1(u):
    return transform_base(u, 0.15, 0.5, water_model='z19')

def transform_h2(u):
    return transform_base(u, 0.5, 1.0, water_model='z19')

### Initialise the log posterior function

In [5]:
names, radii, masses = read_stpm('../../spright/data/stpm_230202.csv')
rme = RMEstimator(nsamples=200, names=names, radii=radii, masses=masses)
lpf = rme.lpf
rdm = lpf.rdm

In [8]:
def lnlikelihood(pv):
    if is_puffy_population_ok(pv, lpf.rdm):
        lnl = lnlikelihood_sample(pv, lpf.density_samples, lpf.radius_samples,
                                  rdm._rr0, rdm._rdr, rdm._rx0, rdm._rdx, rdm.drocky, 
                                  rdm._wr0, rdm._wdr, rdm._wx0, rdm._wdx, rdm.dwater)
        return lnl if isfinite(lnl) else -1e6
    else:
        return -1e6

### Initialise the Dynesty samplers

In [9]:
s0 = NestedSampler(lnlikelihood, transform_h0, 11, nlive=1000)
s1 = NestedSampler(lnlikelihood, transform_h1, 11, nlive=1000)
s2 = NestedSampler(lnlikelihood, transform_h2, 11, nlive=1000)

### Run the samplers and print the evidences

In [10]:
s0.run_nested()

13643it [02:50, 79.86it/s, +1000 | bound: 69 | nc: 1 | ncall: 352545 | eff(%):  4.165 | loglstar:   -inf < -75.441 <    inf | logz: -89.470 +/-  0.153 | dlogz:  0.001 >  1.009]


In [11]:
s1.run_nested()

12328it [03:06, 66.13it/s, +1000 | bound: 60 | nc: 1 | ncall: 310686 | eff(%):  4.304 | loglstar:   -inf < -77.078 <    inf | logz: -89.780 +/-  0.143 | dlogz:  0.001 >  1.009]


In [12]:
s2.run_nested()

13659it [03:33, 63.91it/s, +1000 | bound: 69 | nc: 1 | ncall: 351279 | eff(%):  4.185 | loglstar:   -inf < -77.169 <    inf | logz: -91.184 +/-  0.157 | dlogz:  0.001 >  1.009]


In [13]:
z0 = ufloat(s0.results.logz[-1], s0.results.logzerr[-1])
z1 = ufloat(s1.results.logz[-1], s1.results.logzerr[-1])
z2 = ufloat(s2.results.logz[-1], s2.results.logzerr[-1])

In [18]:
print(2*(z1 - z0), 2*(z2 - z0), 2*(z2 - z1))

-0.6+/-0.7 -3.4+/-0.7 -2.8+/-0.7


---

<center>
©2023 Hannu Parviainen
</center>